In [ ]:
import pandas as pd
import numpy as np
import json
import os

In [ ]:
base_dir = "/path/to/SITC"
subsequences_csv = os.path.join(base_dir, "sequences.csv")

df = pd.read_csv(subsequences_csv, delimiter=';', header=None)

In [ ]:
# set the first row of df.T as the column names
df = df.T
df.columns = df.iloc[0]
df = df.drop(0)

In [ ]:
kitchen_sensors = [1,1,1,2,2,2,1,1,1,1,2,2,1,2,2,1]

activities = [
    "B1_BED_OUT",
    "B2_BED_OUT",
    "B1_JACKET_ON",
    "B2_JACKET_ON",
    "_FRIDGE_OPEN",
    "_FRIDGE_CLOSE",
    "_PREPARE_MEAL",
    "D1_WATER",
    "D2_WATER",
    "D1_EAT",
    "D2_EAT",
    "L1_SIT_DOWN",
    "L2_SIT_DOWN",
    "L1_WATCH_TV",
    "L2_WATCH_TV",
    "L1_STAND_UP",
    "L2_STAND_UP",
    "L1_FALL_DOWN",
    "L2_FALL_DOWN",
    "E1_SHOES_ON",
    "E1_LEAVE_HOUSE",
    "E1_ENTER_HOUSE",
    "E1_SHOES_OFF",
    "W1_BRUSH_TEETH",
    "F1_TAKE_BATH",
    "F1_CLEAN_BATH",
    "B1_JACKET_OFF",
    "B2_JACKET_OFF",
    "B1_BED_IN",
    "B2_BED_IN",
]

In [ ]:
for i, col in enumerate(df.columns):
    person_id = i + 1
    print(f"==================== Person {person_id} ====================")
    for row, activity in enumerate(activities):
        print(f"Activity {activity}")

        if activity.startswith('_'):
            activity = f"K{kitchen_sensors[person_id]}" + activity
        
        sequence_nr = json.loads(df.iloc[row][col])

        if sequence_nr == []:
            continue

        else:
            keypoints_file = os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/pseudocolour_keypoints_interp_spline.json")

            with open(keypoints_file) as f:
                kpts = json.load(f)

            interval_file = os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/subsequences.txt")

            with open(interval_file) as f:
                intervals = f.readlines()

            for seq in sequence_nr:
                print(f"Sequence {seq}")
                interval = intervals[seq].split('\n')
                interval = [x.split(",") for x in interval[0].split(":")][1][:2]
                interval = [int(x.strip()) for x in interval]
                kpts_interval = {k: v for k, v in kpts.items() if interval[0] <= int(k) <= interval[1]}
                print(f"Number of keypoints in interval {seq}: {len(kpts_interval)}")

                if not os.path.exists(os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/converted_kpts/{seq}")):
                        os.makedirs(os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/converted_kpts/{seq}"))

                else:
                    for file_name in os.listdir(os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/converted_kpts/{seq}")):
                        os.remove(os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/converted_kpts/{seq}/{file_name}"))

                for frame_nr, frame in enumerate(kpts_interval.keys()):
                    new_frame = {}
                    new_frame["version"] = 1.2
                    new_frame["people"] = []
                    frame_kpts = kpts_interval[frame][0]
                    
                    new_frame_kpts = {}
                    std_const = frame_kpts[4]
                    std_const[2] = 0

                    new_frame_kpts[0] = np.array(frame_kpts[0]) - np.array(std_const)
                    new_frame_kpts[1] = np.array([(frame_kpts[5][0] + frame_kpts[6][0]) / 2, (frame_kpts[5][1] + frame_kpts[6][1]) / 2, (frame_kpts[5][2] + frame_kpts[6][2]) / 2]) - np.array(std_const)
                    new_frame_kpts[2] = np.array(frame_kpts[6]) - np.array(std_const)
                    new_frame_kpts[3] = np.array(frame_kpts[8]) - np.array(std_const)
                    new_frame_kpts[4] = np.array(frame_kpts[10]) - np.array(std_const)
                    new_frame_kpts[5] = np.array(frame_kpts[5]) - np.array(std_const)
                    new_frame_kpts[6] = np.array(frame_kpts[7]) - np.array(std_const)
                    new_frame_kpts[7] = np.array(frame_kpts[9]) - np.array(std_const)
                    new_frame_kpts[8] = np.array([(frame_kpts[11][0] + frame_kpts[12][0]) / 2, (frame_kpts[11][1] + frame_kpts[12][1]) / 2, (frame_kpts[11][2] + frame_kpts[12][2]) / 2]) - np.array(std_const)
                    new_frame_kpts[9] = np.array(frame_kpts[12]) - np.array(std_const)
                    new_frame_kpts[10] = np.array(frame_kpts[14]) - np.array(std_const)
                    new_frame_kpts[11] = np.array(frame_kpts[16]) - np.array(std_const)
                    new_frame_kpts[12] = np.array(frame_kpts[11]) - np.array(std_const)
                    new_frame_kpts[13] = np.array(frame_kpts[13]) - np.array(std_const)
                    new_frame_kpts[14] = np.array(frame_kpts[15]) - np.array(std_const)
                    new_frame_kpts[15] = np.array(frame_kpts[2]) - np.array(std_const)
                    new_frame_kpts[16] = np.array(frame_kpts[1]) - np.array(std_const)
                    new_frame_kpts[17] = np.array(frame_kpts[4]) - np.array(std_const)
                    new_frame_kpts[18] = np.array(frame_kpts[3]) - np.array(std_const)

                    new_frame_kpts[19] = new_frame_kpts[14]
                    new_frame_kpts[20] = new_frame_kpts[14]
                    new_frame_kpts[21] = new_frame_kpts[14]
                    new_frame_kpts[22] = new_frame_kpts[11]
                    new_frame_kpts[23] = new_frame_kpts[11]
                    new_frame_kpts[24] = new_frame_kpts[11]

                    new_frame_kpts = [item for sublist in new_frame_kpts.values() for item in sublist]
                    
                    new_frame["people"].append({"pose_keypoints_2d": new_frame_kpts, "face_keypoints_2d": [], "hand_left_keypoints_2d": [], "hand_right_keypoints_2d": [], 
                                                "pose_keypoints_3d": [], "face_keypoints_3d": [], "hand_left_keypoints_3d": [], "hand_right_keypoints_3d": []})

                    with open(os.path.join(base_dir, f"Person{str(person_id).zfill(3)}/{activity}/converted_kpts/{seq}/{frame_nr}.json"), "w") as f:
                        json.dump(new_frame, f)

    print("=========================================================")